In [1]:
import geopandas as gpd
from glob import glob

from rasterio.features import rasterize
from rasterio.io import MemoryFile
import rasterio.mask
import rasterio

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
avoid_labels = {'SNE': 'Terreno Inutilizzato',

                'PPH': 'Prato Permanente',

                'SPH': 'Superficie pastorale',
                'SPL': 'Superficie pastorale',
                
                'J6P': 'Maggese - rotazione lunga',
                'J6S': 'Maggese - rotazione media',
                'J5M': 'Maggese - temporaneo',

                'PRL': 'Prato - rotazione lunga',
                'PTR': 'Prato - temporaneo',

                'LUZ': 'Erba Medica', 
                
                'BOP': 'Bosco',
                'TRE': 'Altro Trifoglio',
                'BOR': 'Confine del campo',     
               }


valid_labels = {'VRC': 'Vitigno - Produttivo',
                'VRN': 'Vitigno - Non  Produttivo',

                'BTH': 'Grano tenero invernale',
                'BDH': 'Grano duro invernale',

                'MIS': 'Mais',
                'TRN': 'Girasole',
                
                'ORH': 'Orzo Invernale', 
                'ORP': 'Orzo Primaverile',

                'CZH': 'Colza Invernale',
                
                'MIE': 'Insilato di mais',
                'TTH': 'Triticale Invernale',

                'RGA': 'Loglio - temporaneo',
                'MLG': 'Miscela di legumi',
                'MCR': 'Cereali Misti', 
                
                'VRG': 'Frutteto',
                'TRN': 'Girasole',
 
                'SOG': 'Sorgo',
                'SOJ': 'Soia',
                'NOX': 'Noce',
                'SGH': 'Segale Invernale',
                
                'MLF': 'Misto di Legumi da Foraggio',
                'BTA': 'Fascia Tampone' }

labels = {**valid_labels, **avoid_labels}

vocabulary = {v:i+1 for i, v in enumerate(labels)}

#### Tiles Selection
- Extract information (polygons) of the tiles included into the anlysis. 

In [4]:
gdf = gpd.read_file(glob('../../data/Sentinel-2-Shapefile-Index/*.shp')[1])
gdf = gdf.to_crs('epsg:4326').set_index('Name')

In [5]:
tiles = ['30TYR', '30TYP', '31TCJ']

In [6]:
from shapely.ops import unary_union
boundary = unary_union(gdf.loc[tiles].geometry)

#### Parcel Selection Rules:
- Select only parcels greather than 2500 m2
- Select only parcels that are within the selected tiles

In [7]:
lower_area_bound = (50. * 50.) / 10000.

In [8]:
parcels = gpd.read_file('../../data/Parcels/Parcels_South_France.geojson')

In [9]:
parcels = parcels[parcels.CODE_CULTU.isin(labels.keys())]
parcels = parcels[parcels['SURF_PARC'] > lower_area_bound]

In [10]:
parcels = parcels[parcels.geometry.apply(lambda x: x.intersects(boundary))]

In [11]:
parcels['CODE_GROUP'] = parcels.CODE_CULTU.map(vocabulary)

In [12]:
crops = pd.concat([gpd.read_file('/extra/Sentinel-2-crops/T{}/metadata.json'.format(tile)).to_crs('epsg:4326') 
                   for tile in ['30TYR', '30TYP', '31TCJ']])

In [13]:
crops = crops.reset_index()

In [14]:
profile = {'driver': "GTiff", 'height': 128, 'width': 128, 
           'dtype': 'uint16', 'nodata': None,'crs': "EPSG:4326",
           'tiled': True}

In [15]:
def create_mask(crop):
    transform = rasterio.transform.from_bounds(*(crop.geometry.bounds + (128, 128)))
    mfile = MemoryFile().open(**{**profile, **{'transform': transform, 'count': 1}})
    targets = parcels[parcels.geometry.apply(lambda x: crop.geometry.intersects(x))]

    ### Coverage condition:
    if targets.SURF_PARC.sum() > (100.*100. / 10000.):
        mask  = np.zeros((128, 128))
        check = np.zeros((128, 128))
        for _, target in targets.iterrows():
            aux = rasterio.mask.raster_geometry_mask(mfile, [target.geometry], 
                                                     crop=False, invert=True)
            
            mask  += aux[0].astype(int) * target['CODE_GROUP']
            check += aux[0].astype(int)
        
        mask[check > 1] = 0
        
        filename = '/extra/Sentinel-2-crops/{}/TARGETS/TARGET_{}.npy'
        np.save(open(filename.format(crop.tile, crop.id), 'wb'), mask)
        
        filename = '/extra/Sentinel-2-crops/{}/{}/*.npy'
        files = glob(filename.format(crop.tile, crop.id))
        dates = {str(i):f.split('/')[-1][:-4] for i,f in enumerate(files)}
        
        return {
                    'Fold':crop['index']%5+1, 
                    'id':crop['id'],
                    'ID_PATCH':'{}-{}'.format(crop.tile.lower(), crop['index']),
                    'N_Parcel': len(targets),
                    'Parcel_Cover': (mask!=0).sum() / (128*128), 
                    'TILE':crop.tile.lower(),
                    'dates-S2': dates, 
                    'geometry':crop.geometry
               }

In [ ]:
out = crops.apply(create_mask, axis=1)

> /tmp/ipykernel_921/4111273383.py(27)create_mask()
     25 
     26         return {
---> 27                     'Fold':crop['index']%5+1,
     28                     'id':crop['id'],
     29                     'ID_PATCH':'{}-{}'.format(crop.tile.lower(), crop['index']),



ipdb>  mask.max()


34.0


ipdb>  check.max()


2.0


...

21


ipdb>  parcels.CODE_GROUP


900506      7
900507      7
900508      7
900596      3
900617     28
           ..
3135678    23
3135679    23
3135680    23
3135681    27
3135682    27
Name: CODE_GROUP, Length: 587777, dtype: int64


ipdb>  parcels.CODE_GROUP.max()


34


In [ ]:
out = gpd.GeoDataFrame(out.dropna().tolist())
out.to_file('../../data/Crops/metadata.geojson')